### This Notebook is an attempt to create a GUI through Python.
**I am going to be sincere and say that this work is all sweat and tears from Chat GPT**

In [5]:
## ideas to incorporate!
## ctrl z button
## a next button: you load a file with a number of profiles in it and then you can flick through them one by one 


In [ ]:
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

In [28]:
# Generate dummy data for temperature profiles
num_profiles = 2
depth = np.arange(0, 1000, 10)  # Depth in meters
temperature_profiles = []
for _ in range(num_profiles):
    temperature = np.random.uniform(2, 20, len(depth))  # Temperature in °C
    temperature_profiles.append((depth, temperature))


In [29]:
temperature_profiles

[(array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
         130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
         260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
         390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
         520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640,
         650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770,
         780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900,
         910, 920, 930, 940, 950, 960, 970, 980, 990]),
  array([12.85965985, 19.70761845, 14.29037466,  6.41597951, 19.22702621,
         10.11161298, 18.52079776, 13.98503482, 17.42342361, 12.09847112,
         12.66706762, 12.06480963, 16.52524889, 12.60356293, 14.82812614,
         19.16743702, 10.11067317,  6.22096844,  4.89081016,  3.50567595,
          7.46008283,  5.7201969 , 18.4530084 , 19.91544159, 10.93229413,
         15.00994362,  3.13194789,  2.96864621, 16.57108

In [30]:
%matplotlib qt

class OceanProfileGUI:
    def __init__(self, root, temperature_profiles):
        self.root = root
        self.root.title("Ocean Profile Analysis")

        self.temperature_profiles = temperature_profiles
        self.current_profile_index = 0  # Index of the currently displayed profile

        self.canvas = plt.figure(figsize=(6, 4))
        self.plot_area = self.canvas.add_subplot(111)
        self.plot_area.set_xlabel("Temperature (°C)")
        self.plot_area.set_ylabel("Depth (m)")

        self.canvas_widget = FigureCanvasTkAgg(self.canvas, master=root)
        self.canvas_widget.get_tk_widget().pack()

        self.selected_depth = None

        self.load_profile()  # Load first profile initially
        self.next_button = tk.Button(root, text="Next", command=self.load_next_profile)
        self.next_button.pack()

        self.canvas_widget.mpl_connect('button_press_event', self.on_click)

    def load_profile(self):
        # Plot the current profile
        self.plot_area.clear()
        depth, temperature = self.temperature_profiles[self.current_profile_index]
        self.plot_area.plot(temperature, depth)
        self.canvas_widget.draw()

    def load_next_profile(self):
        # Load and plot the next profile
        self.current_profile_index = (self.current_profile_index + 1) % len(self.temperature_profiles)
        self.load_profile()

    def on_click(self, event):
        # Get depth where user clicked
        depth = event.ydata
        # Record selected depth
        self.selected_depth = depth
        # Mark selected depth on the plot
        self.plot_area.axhline(depth, color='r', linestyle='--')
        self.canvas_widget.draw()

# Create the GUI
root = tk.Tk()
gui = OceanProfileGUI(root, temperature_profiles)
root.mainloop()

In [31]:
%matplotlib qt
class OceanProfileGUI:
    def __init__(self, root, temperature_profiles):
        self.root = root
        self.root.title("Ocean Profile Analysis")

        self.temperature_profiles = temperature_profiles
        self.num_profiles = len(temperature_profiles)
        self.current_profile_index = 0  # Index of the currently displayed profile

        self.canvas = plt.figure(figsize=(6, 4))
        self.plot_area = self.canvas.add_subplot(111)
        self.plot_area.set_xlabel("Temperature (°C)")
        self.plot_area.set_ylabel("Depth (m)")

        self.canvas_widget = FigureCanvasTkAgg(self.canvas, master=root)
        self.canvas_widget.get_tk_widget().pack()

        self.selected_depth = None
        self.recorded_depths = []  # List to store recorded depths

        self.load_profile()  # Load first profile initially
        self.next_button = tk.Button(root, text="Next", command=self.load_next_profile)
        self.next_button.pack()

        self.profile_label = tk.Label(root, text=f"Profiles: {self.num_profiles}")
        self.profile_label.place(x=10, y=10)  # Position the label on the top-left corner

        self.canvas_widget.mpl_connect('button_press_event', self.on_click)

    def load_profile(self):
        # Plot the current profile
        self.plot_area.clear()
        depth, temperature = self.temperature_profiles[self.current_profile_index]
        self.plot_area.plot(temperature, depth)
        self.canvas_widget.draw()

    def load_next_profile(self):
        # Load and plot the next profile
        self.current_profile_index = (self.current_profile_index + 1) % self.num_profiles
        self.load_profile()

    def on_click(self, event):
        # Get depth where user clicked
        depth = event.ydata
        # Record selected depth
        self.selected_depth = depth
        self.recorded_depths.append(depth)  # Record depth
        # Mark selected depth on the plot
        self.plot_area.axhline(depth, color='r', linestyle='--')
        self.canvas_widget.draw()

if __name__ == "__main__":
    # Assuming temperature_profiles is the variable containing the loaded profiles

    # Create the GUI
    root = tk.Tk()
    gui = OceanProfileGUI(root, temperature_profiles)
    root.mainloop()

    # Output recorded depths
    print("Recorded Depths:", gui.recorded_depths)

Recorded Depths: [325.28571428571433, 483.2142857142858, 556.2857142857143, 650.5714285714286, 551.5714285714286]


In [32]:
pwd

'C:\\Users\\nribeiro\\OneDrive - University of Tasmania\\IMOS Shared Docs'